## Análise de Cotações — Sumário em Tópicos

- **Escopo**: consolidação diária por `cod_negociacao` e análises aplicadas de integridade, outliers, liquidez, retornos, risco e dependências.
- **Integridade dos dados**:
  - Consistência de preços: `preco_maximo ≥ preco_minimo`, `preco_medio` entre `[min,max]`.
  - Sinais inválidos: negativos e `preco_fechamento == 0`.
- **Outliers**:
  - Preço: flag para `preco_medio > 100.000` e `< 0,01`.
  - Volume: flag por quantil alto `p99.9`.
- **Liquidez por ticker**:
  - Medianas de `qtd_negocios` e `volume` por dia.
  - Seleção de `top_liquidos` para reduzir ruído e custo computacional.
- **Retornos e “mercado”**:
  - `retorno = pct_change(fechamento)` por ticker.
  - Índice equal-weight diário em tickers mais líquidos (`retorno_mkt`).


In [1]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('datasets/cotacoes_resumidas_2018.csv')

df.head()

,data_pregao,cod_negociacao,tipo_mercado,nome_empresa,especificacao,moeda,preco_abertura,preco_maximo,preco_minimo,preco_medio,preco_fechamento,qtd_negocios,volume
0,2018-01-02,AALR3,10,ALLIAR,ON NM,R$,14.94,15.16,14.70,14.84,14.89,403,1402991
1,2018-01-02,AALR3F,20,ALLIAR,ON NM,R$,14.79,14.93,14.79,14.86,14.80,7,3343
2,2018-01-02,AAPL34,10,APPLE,DRN,R$,56.81,56.81,56.30,56.38,56.30,4,50747
3,2018-01-02,AAPL34F,20,APPLE,DRN,R$,56.81,65.00,56.54,59.63,56.54,7,3637
4,2018-01-02,ABCB2,10,ABC BRASIL,DIR PRE N2,R$,4.11,4.20,4.11,4.14,4.20,6,4977


## Análise Superficial

In [9]:
df.isnull().sum()

data_pregao         0
cod_negociacao      0
tipo_mercado        0
nome_empresa        0
especificacao       0
moeda               0
preco_abertura      0
preco_maximo        0
preco_minimo        0
preco_medio         0
preco_fechamento    0
qtd_negocios        0
volume              0
dtype: int64

In [10]:
df.dtypes

data_pregao          object
cod_negociacao       object
tipo_mercado          int64
nome_empresa         object
especificacao        object
moeda                object
preco_abertura      float64
preco_maximo        float64
preco_minimo        float64
preco_medio         float64
preco_fechamento    float64
qtd_negocios          int64
volume                int64
dtype: object

In [11]:
df.describe()


,tipo_mercado,preco_abertura,preco_maximo,preco_minimo,preco_medio,preco_fechamento,qtd_negocios,volume
count,580155.000000,580155.000000,580155.000000,580155.000000,580155.000000,580155.000000,580155.000000,5.801550e+05
mean,48.310943,100.388153,101.500984,99.297951,100.327645,100.386051,529.126721,5.198086e+06
std,29.189584,1600.478371,1601.702154,1599.389329,1600.421272,1600.556509,2894.814529,5.604711e+07
min,10.000000,0.010000,0.010000,0.010000,0.010000,0.010000,1.000000,0.000000e+00
25%,20.000000,0.600000,0.650000,0.550000,0.600000,0.600000,2.000000,1.888000e+03
50%,70.000000,2.680000,2.790000,2.580000,2.670000,2.680000,5.000000,1.595100e+04
75%,70.000000,18.880000,19.030000,18.600000,18.830000,18.860000,34.000000,1.423740e+05
max,80.000000,110000.000000,110000.000000,110000.000000,110000.000000,110000.000000,97176.000000,1.077648e+10


In [12]:

y = df["preco_medio"]

print("Tem NaN?", y.isna().sum())
print("Tem inf?", np.isinf(y).sum())
print("Valores > 100_000:", (y > 100_000).sum())  # >100 mil pode ser fora do padrão

Tem NaN? 0
Tem inf? 0
Valores > 100_000: 2


identificamos valores acima de 100.000 como preço médio da ação, o que não faz sentido. Vamos remover esse tipo de outlier posteriormente

In [13]:
df[df['preco_medio'] > 100000]['cod_negociacao'].value_counts()

cod_negociacao
IBOVP110E    1
IBOVR110E    1
Name: count, dtype: int64

### Integridade e outliers — como ler
- **Objetivo**: validar consistência dos preços e sinalizar valores suspeitos.
- **Checagens**: `preco_maximo ≥ preco_minimo`, `preco_medio ∈ [min,max]`, negativos e fechamento igual a zero.
- **Outliers**: `preco_medio > 100.000` ou `< 0,01`; volume acima do `p99.9`.
- **Ação sugerida**: filtrar/ajustar outliers antes de modelagem; revisar tickers com maior incidência.


In [14]:
# Regras de integridade de preços
viol_max_min = (df['preco_maximo'] < df['preco_minimo']).sum()
viol_media_fora = ((df['preco_medio'] < df['preco_minimo']) | (df['preco_medio'] > df['preco_maximo'])).sum()
negativos = (df[['preco_abertura','preco_maximo','preco_minimo','preco_medio','preco_fechamento','qtd_negocios','volume']] < 0).any(axis=1).sum()
zeros_close = (df['preco_fechamento'] == 0).sum()

print('max < min:', viol_max_min)
print('médio fora do intervalo:', viol_media_fora)
print('valores negativos em colunas-chave:', negativos)
print('fechamento igual a zero:', zeros_close)


max < min: 0
médio fora do intervalo: 3
valores negativos em colunas-chave: 0
fechamento igual a zero: 0


In [15]:
# Outliers de preço e volume
out_preco_alto = df['preco_medio'] > 100_000
out_preco_baixo = df['preco_medio'] < 0.01
# Usa quantil alto como referência de outlier de volume sem depender de valor absoluto
o_utlimo_quantil = df['volume'].quantile(0.999)
out_volume_alto = df['volume'] > o_utlimo_quantil

print('linhas com preco_medio > 100k:', int(out_preco_alto.sum()))
print('linhas com preco_medio < 0.01:', int(out_preco_baixo.sum()))
print('linhas com volume acima do p99.9%:', int(out_volume_alto.sum()))

if out_preco_alto.any():
    top_tickers_out = df.loc[out_preco_alto, 'cod_negociacao'].value_counts().head(15)
    top_tickers_out


linhas com preco_medio > 100k: 2
linhas com preco_medio < 0.01: 0
linhas com volume acima do p99.9%: 581


### Liquidez — critérios e leitura
- **Agregação diária**: soma de `qtd_negocios` e `volume`; `fechamento` do dia.
- **Métricas**: medianas por ticker (`qtd_negocios`, `volume`) e nº de dias negociados.
- **Seleção**: `top_liquidos` (100) para reduzir ruído e custo nas análises seguintes.
- **Interpretação**: maiores medianas → maior negociabilidade/qualidade de preço.


In [16]:
# Métricas de liquidez por ticker (agregação diária)
# Construímos um dataset diário por ticker para facilitar análises e reduzir custo computacional

diaria = (df.groupby(['cod_negociacao','data_pregao'], as_index=False)
            .agg(qtd_negocios=('qtd_negocios','sum'),
                 volume=('volume','sum'),
                 fechamento=('preco_fechamento','last'),
                 maximo=('preco_maximo','max'),
                 minimo=('preco_minimo','min')))

liq = (diaria.groupby('cod_negociacao')
            .agg(mediana_negocios=('qtd_negocios','median'),
                 mediana_volume=('volume','median'),
                 dias=('data_pregao','nunique'))
            .sort_values(['mediana_negocios','mediana_volume'], ascending=False))

top_liquidos = liq.head(100).index.to_list()
liq.head(15)


,mediana_negocios,mediana_volume,dias
cod_negociacao,,,
PETR4,48855.0,1.196356e+09,245
VALE3,30211.0,8.066895e+08,245
ITUB4,28371.0,5.726220e+08,245
BBDC4,27020.0,3.980033e+08,245
ITSA4,25518.0,2.020711e+08,245
ABEV3,24720.0,2.785946e+08,245
B3SA3,23559.0,2.202551e+08,189
BBAS3,22602.0,3.697146e+08,245
BVMF3,21418.5,2.134370e+08,56


### Retornos e mercado equal-weight
- **Cálculo**: `retorno = pct_change(fechamento)` por `cod_negociacao`.
- **Mercado (EW)**: média simples dos retornos dos tickers em `top_liquidos` por dia.
- **Uso**: base para análise de extremos, volatilidade e correlações.
- **Observação**: verifique gaps/zeros antes de interpretar retornos.


In [17]:
# Retornos diários por ticker a partir do fechamento

diaria = diaria.sort_values(['cod_negociacao','data_pregao'])
diaria['retorno'] = diaria.groupby('cod_negociacao')['fechamento'].pct_change()

# Retorno de mercado equal-weight usando apenas tickers mais líquidos (reduz ruído)
retornos_liquidos = diaria[diaria['cod_negociacao'].isin(top_liquidos)]
mkt = (retornos_liquidos
       .dropna(subset=['retorno'])
       .groupby('data_pregao')['retorno']
       .mean()
       .to_frame('retorno_mkt')
       .reset_index())

mkt.describe()


,retorno_mkt
count,244.000000
mean,0.000973
std,0.016277
min,-0.053130
25%,-0.009062
50%,0.000931
75%,0.009633
max,0.059295


### Interpretação e leitura
- **Integridade**: se houver `max < min`, `médio` fora de `[min,max]`, negativos ou `fechamento == 0`, trate como dado inconsistente.
- **Outliers**: valores marcados (preço/volume) podem distorcer estatísticas; considere excluir ou winsorizar.
- **Liquidez**: use `mediana_negocios` e `mediana_volume` para priorizar ativos com melhor formação de preço.
- **Retornos**: calcule a partir do `fechamento`; interprete o índice equal-weight como proxy do “mercado”.

### Próximos passos
Na treinamento vamos aplicar transformações e tratamento dos dados para reduzir o impacto de outliers e dados sem sentido